In [82]:
#Importing
import dictdiffer
import pandas as pd
from copy import deepcopy
import matplotlib.pyplot as plt
from pandas.plotting import table

<h3>Desc</h3>
<p>Perahu bisa tampung petani & 1 barang lain (Kambing, Serigala, Sayuran)<br>
<h4>Rules:</h4>
<ul>
<p>Jika ditinggal petani, maka :</p>
<li>1. Sayuran Dimakan Kambing</li>
<li>2. Kambing Dimakan Serigala</li>
</ul>
</p>
<h4>Aturan Penyeberangan:</h4>
<ul>
<li>1. Kambing menyeberang</li>
<li>2. Sayuran menyeberang</li>
<li>3. Serigala menyeberang</li>
<li>4. Kambing kembali</li>
<li>5. Sayuran kembali</li>
<li>6. Serigala kembali</li>
<li>7. Perahu kembali</li>
</ul>
<h4>Kondisi awal</h4>
<p>
(Jumlah Kambing, Jumlah Serigala, Jumlah Sayuran, Jumlah Perahu)<br>
Example: (1,1,1,1)
</p>

<h3>Init state</h3>

In [83]:
START_STATE = {'kambing': 'asal', 'serigala': 'asal', 'sayuran': 'asal', 'perahu': 'asal'}
FINAL_STATE = {'kambing': 'tujuan', 'serigala': 'tujuan', 'sayuran': 'tujuan', 'perahu': 'tujuan'}

<h3>Aturan-aturan</h3>

In [84]:
ATURAN = [
    {'perahu': 'tujuan', 'kambing': 'tujuan'},  # 1. perahu membawa kambing ke tujuan
    {'perahu': 'tujuan', 'sayuran': 'tujuan'},  # 2. perahu membawa sayuran ke tujuan
    {'perahu': 'tujuan', 'serigala': 'tujuan'},  # 3. perahu membawa serigala ke tujuan
    {'perahu': 'asal', 'kambing': 'asal'},  # 4. perahu membawa kambing ke asal
    {'perahu': 'asal', 'sayuran': 'asal'},  # 5. perahu membawa sayuran ke asal
    {'perahu': 'asal', 'serigala': 'asal'},  # 6. perahu membawa serigala ke asal
    {'perahu': 'asal'},  # 7. perahu kembali ke asal
    # {'perahu': 'tujuan'}  # 8. perahu jalan ke tujuan
]

<h3>Cek apakah kondisi memenuhi rule</h3>

In [85]:
def is_valid(current_state:dict) -> bool:
    #Cek apakah serigala dan kambing berada di sisi yang sama
    if current_state['serigala'] == current_state['kambing'] and current_state['perahu'] != current_state['serigala']:
    # if current_state['serigala'] == current_state['kambing']:
        return False
    #Cek apakah kambing dan sayuran berada di sisi yang sama
    if current_state['kambing'] == current_state['sayuran'] and current_state['perahu'] != current_state['kambing']:
    # if current_state['kambing'] == current_state['sayuran']:
        return False
    return True

<h3>BFS Algorithm</h3>

In [86]:
def bfs(state:dict) -> list:
    queue = []
    queue.append((state, [0]))
    while queue:
        current_state, path = queue.pop(0)
        # print(*queue)
        if current_state == FINAL_STATE:
            return path
        for key,action in enumerate(ATURAN):
            new_state = {**current_state, **action}
            if is_valid(new_state):
            # if is_valid(new_state):
                # new_path = path + [new_state]
                new_path = path
                new_path.append(key+1)
                queue.append((new_state, new_path))
    return None

In [87]:
def bfs2(state: dict):
    state['tingkatan_tree'] = '0'
    df = pd.DataFrame(columns=state.keys())
    df = df.append(state, ignore_index=True)
    queue = []
    tingkatan_tree = 0
    queue.append((state, []))
    while queue:
        current_state, path = queue.pop(0)
        tingkatan_tree += 1
        del current_state['tingkatan_tree']
        if current_state == FINAL_STATE:
            # return path + [current_state]
            current_state['tingkatan_tree'] = str(tingkatan_tree)
            df = df.append(current_state, ignore_index=True)
            return df
        current_state['tingkatan_tree'] = str(tingkatan_tree)
        for num,action in enumerate(ATURAN):
            #Current state jadi acuan
            new_state = current_state.copy()
            cont_state = 0
            list_action = list(action.keys())
            list_action_value = list(action.values())
            for key,value in zip(list_action,list_action_value):
                if(current_state[key] == value):
                    cont_state = 1
                    break
                else:
                    new_state[key] = value
            if(cont_state):
                continue
            # print(ATURAN[num])
            # new_state = {**current_state, **action}
            if is_valid(new_state):
                new_path = path
                new_path.append(action)
                queue.append((new_state, new_path))
                df = df.append(new_state, ignore_index=True)
    return None


In [88]:

def get_bfs():
    bfs_hasil = bfs2(START_STATE)

    ax = plt.subplot(111,frame_on=False)
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    table(ax,bfs_hasil,loc='center')
    plt.show()

<h3>DFS Algorithm</h3>

In [89]:
def apply_rule(state: dict, rule: dict) -> dict:
    new_state = state.copy()
    for key, value in rule.items():
        new_state[key] = value
    return new_state

def dfs(start_state: dict, final_state: dict) -> list:
    stack = [(start_state, [])]
    visited = set()
    while stack:
        state, path = stack.pop()
        if state == final_state:
            return path + [state]
        visited.add(tuple(state.items()))
        for action in ATURAN:
            new_state = apply_rule(state, action)
            if tuple(new_state.items()) not in visited and is_valid(new_state):
                stack.append((new_state, path + [state]))
    return None

In [90]:
# def dfs(current_state, visited):
#     if current_state == FINAL_STATE:
#         return [current_state]

#     visited.add(frozenset(current_state.items()))

#     for aturan in ATURAN:
#         new_state = current_state.copy()
#         new_state.update(aturan)

#         if frozenset(new_state.items()) in visited:
#             continue

#         if is_valid(new_state):
#             solution = dfs(new_state, visited)
#             if solution is not None:
#                 return [current_state] + solution

#     return None


# def dfs_solve(start_state):
#     visited = set()
#     return dfs(start_state, visited)

<h3>Print Hasil</h3>

In [91]:
def get_dfs():
    path = dfs(START_STATE, FINAL_STATE)
    print(*path,sep='\n')

In [94]:
get_dfs()

TypeError: print() argument after * must be an iterable, not NoneType